Базовый эксперимент с RNN, без вариационного вывода

In [1]:
import numpy as np
import theano
import theano.tensor as T
from sklearn.metrics import f1_score
from collections import OrderedDict

Using gpu device 0: GeForce 820M (CNMeM is disabled, cuDNN not available)


NB: пока что повторяем статью, поэтому делим выборку в отношении 70%/15%/15%.

Валидационную выборку (вторая часть) можно использовать для контроля переобучения.

По поводу переменных Mask1, Mask2, M_train, и и т.д. 

Мы хотим, чтобы наша рекуррентная функция работала достаточно быстро, поэтому будем подавать на вход сразу батч. В связи с этим возникает проблема: все элементы батча должны иметь одинаковые размерности, а значит коротки предложения нужно по длине доводить до длинных. Предлагается создавать большой нулевой тензор, забивать его нашими предложениями, а то, что не заполнено предложениями помечать в маске нулями.



In [29]:
X = np.load('semeval_x.npy')
Y= np.load('semeval_y.npy').astype(np.int32)
part = int(len(Y)*0.7)
part2 = int(len(Y)*0.85)

max_len = max([len(x) for x in X])
X_masked1,X_masked2  = np.zeros((Y.shape[0], max_len, 50)).astype(theano.config.floatX),np.zeros((Y.shape[0], max_len, 50)).astype(theano.config.floatX)
Mask1, Mask2 = np.zeros((Y.shape[0], max_len)).astype(theano.config.floatX), np.zeros((Y.shape[0], max_len)).astype(theano.config.floatX)
for i in xrange(X.shape[0]/2):
    X_masked1[i, :len(X[2*i])] = X[2*i]
    X_masked2[i, :len(X[2*i+1])] = X[2*i+1]
    Mask1[i,  :len(X[2*i])] = 1.0
    Mask2[i,  :len(X[2*i+1])] = 1.0
    
X_train1,  X_valid1, X_test1=X_masked1[:part], X_masked1[part:part2], X_masked1[part2:],
X_train2, X_valid2, X_test2 = X_masked2[:part], X_masked2[part:part2], X_masked2[part2:]

M_train1, M_valid1, M_test1= Mask1[:part], Mask1[part:part2], Mask1[part2:],
M_train2, M_valid2, M_test2  = Mask2[:part], Mask1[part:part2], Mask2[part2:]
Y_train,  Y_valid, Y_test = Y[:part], Y[part:part2], Y[part2:]



In [30]:
lr = 0.01
lambda2 = 0.01
batch_size = 25
hidden_size = 50
input_size = 50
epoch_num = 1000

Создание входных тензоров и матриц. Если хотим адаптировать под GPU, нужно переписать входы под shared-переменны, смотри документацию теано.

In [31]:
X_tensor1, X_tensor2 = T.tensor3(), T.tensor3()
Mask_matrix1, Mask_matrix2 = T.matrix(), T.matrix()
W = theano.shared((np.random.randn(input_size, hidden_size)*0.01).astype(theano.config.floatX))
U = theano.shared((np.random.randn(hidden_size, hidden_size)*0.01).astype(theano.config.floatX))
b = theano.shared((np.zeros(hidden_size)).astype(theano.config.floatX))
h0 = theano.shared((np.zeros(hidden_size)).astype(theano.config.floatX))
softmax_W = theano.shared((np.random.randn(2*hidden_size,6)*0.01).astype(theano.config.floatX))
softmax_b = theano.shared((np.zeros(6).astype(theano.config.floatX)))
Y_vector = T.ivector()

Один шаг кодирования:
    $$h_i = \text{tanh}(\mathbf{XW} + \mathbf{h_{i-1}U} + \mathbf{b}) $$
    
XW уже перемножено для оптимизации.

In [32]:
def encode_step(XW_matrix, hidden):    
    return T.tanh(XW_matrix  + T.dot(hidden, U) + b) 

Обертка кодирования с учетом маски.

In [33]:
def masked_step( XW, hidden,   mask):

        hid_new = encode_step(XW, hidden)
        mask_axis = T.tile(mask, (XW.shape[1], 1)).T
        hid_out = T.switch(mask_axis, hid_new, hidden)
        return hid_out

In [8]:
train_results = [] #valid_loss + test_loss

Построение модели.

Если мы посмотрим различные обучалки по теано, то там для создания рекурсии используется функуция scan, отвечающая за циклы. Проблема этой функции в том, что работает она очень медленно (это одно из немногих выражений theano, которое не компилируется в C-код, а остается питоновым), поэтому по возможности от нее надо отказываться. 

Если длины всех наших последовательностей небольшие, то можно создать явно выражения для каждого шага рекурсии. Это называет unrolling scan, смотри код RNN в Lasagne и Keras.

In [34]:
XW1 = T.dot(X_tensor1, W)
XW2 = T.dot(X_tensor2, W)
h0_tiled = T.tile(h0,( X_tensor1.shape[0], 1))
hiddens1, hiddens2 = [h0_tiled], [h0_tiled]
for i in xrange(max_len):
    hiddens1.append(masked_step(XW1[:,i,:], hiddens1[-1],  Mask_matrix1[:,i]))
    hiddens2.append(masked_step(XW2[:,i,:], hiddens2[-1],  Mask_matrix2[:,i]))
output = T.nnet.softmax(T.dot(T.concatenate([hiddens1[-1], hiddens2[-1]], axis=1 ), softmax_W) + softmax_b)


In [35]:
softmax_cost = -T.mean(T.log(output)[T.arange(Y_vector.shape[0]), Y_vector])
all_params = T.concatenate([W.flatten(), U.flatten(), b, h0, softmax_W.flatten(), softmax_b])
l2_cost = lambda2*(all_params**2).sum()
cost = softmax_cost + l2_cost

Код adagrad из lasagne

In [36]:
def lasagne_adagrad(loss, params, learning_rate=1.0, epsilon=1e-6):   
    grads = [T.grad(loss,p) for p in params]
    updates = OrderedDict()

    for param, grad in zip(params, grads):
        value = param.get_value(borrow=True)
        accu = theano.shared(np.zeros(value.shape, dtype=value.dtype),
                             broadcastable=param.broadcastable)
        accu_new = accu + grad ** 2
        updates[accu] = accu_new
        updates[param] = param - (learning_rate * grad /
                                  T.sqrt(accu_new + epsilon))

    return updates

Компиляция

In [40]:
train_fn = theano.function([X_tensor1, X_tensor2, Mask_matrix1, Mask_matrix2,  Y_vector], cost,  
                           updates=lasagne_adagrad(cost, [W,U,h0,b,softmax_W,softmax_b], learning_rate=lr), 
                           on_unused_input='ignore' )
predict_fn = theano.function([X_tensor1, X_tensor2, Mask_matrix1, Mask_matrix2], output, on_unused_input='ignore')


Валидация будет происходить на каждой эпохе.
Выбирается случайная сбалансированная подвыборка из валидационной и тестовой и считается их macro-мера.

Вообще мы не должны знать меру на тесте, и не имеем право останавливать оптимизацию, мониторя ее, но на всякий случай будем ее выводить :)

Извиняюсь за корявый код, но мне лень его переписывать.

In [41]:
from random import sample
def my_eval():
    min_class = min([len(np.where(Y_valid==i)[0]) for i in range(6)])
    results = []
    for _ in xrange(10):
        idx = []
        for i in range(6):
            class_idx = np.where(Y_valid==i)[0].tolist()
            idx.extend(sample(class_idx, min_class))
        pred = np.argmax(predict_fn(X_valid1[idx], X_valid2[idx], M_valid1[idx], M_valid2[idx]), axis=1)
        results.append(f1_score(Y_valid[idx], pred, average='macro'))
    print np.mean(results), np.std(results)
    train_results.append([np.mean(results)])
    
    
    min_class = min([len(np.where(Y_test==i)[0]) for i in range(6)])
    results = []
    for _ in xrange(10):
        idx = []
        for i in range(6):
            class_idx = np.where(Y_test==i)[0].tolist()
            idx.extend(sample(class_idx, min_class))
        pred = np.argmax(predict_fn(X_test1[idx], X_test2[idx], M_test1[idx], M_test2[idx]), axis=1)
        results.append(f1_score(Y_test[idx], pred, average='macro'))
    print np.mean(results), np.std(results)
    train_results[-1].extend([np.mean(results)])
    

Непосредственно запуск оптимизации

In [42]:
for epoch in xrange(1000):
    print 'Epoch', epoch
    for batch_start in xrange(0, X_train1.shape[0] + batch_size,  batch_size):
        batch_ids = range(batch_start, min(X_train1.shape[0], batch_start+batch_size))
        if len(batch_ids)==0:
            break
        x1 = X_train1[batch_ids]
        x2 = X_train2[batch_ids]
        m1 = M_train1[batch_ids]
        m2 = M_train2[batch_ids]
        y = Y_train[batch_ids]        
        train_fn(x1,x2, m1, m2, y)
    y_pred = np.argmax(predict_fn(X_train1, X_train2, M_train1, M_train2), axis=1)
    print f1_score(Y_train, y_pred, average='macro')
    my_eval()
    

Epoch 0
0.312814969094
0.276164924122 0.0154397365932
0.287466712355 0.0117645520581
Epoch 1
0.314604097016
0.271011688317 0.0065229095728
0.298339118895 0.0114110170914
Epoch 2
0.316842569246
0.272443458924 0.00528693699661
0.294208080282 0.0115457123869
Epoch 3
0.316882700757
0.275127265524 0.0123200360437
0.297908279441 0.0113198277125
Epoch 4
0.318642287175
0.285822037608 0.00941776767379
0.294364000842 0.00915069121403
Epoch 5
0.31935301272
0.282242878086 0.0141572729915
0.300687988854 0.0159574963594
Epoch 6
0.321097711712
0.277255952173 0.0103621563342
0.294983464762 0.00780157412243
Epoch 7
0.32147950996
0.278844476729 0.00988490636324
0.295672036813 0.0124321133166
Epoch 8
0.320251870747
0.284227229192 0.0108571082292
0.295089764985 0.0106672806514
Epoch 9
0.321022031031
0.288295927279 0.0201845904103
0.298413237686 0.0176232573179
Epoch 10
0.321027805452
0.280207111638 0.0114873878551
0.300069688139 0.00909387720878
Epoch 11
0.321414539146
0.280484992399 0.00789648281066
0.29